In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Define Problem

In [2]:
import numpy as np
from scipy.linalg import null_space

class nmRosenbrock(nn.Module):
    """
    Penalty loss function for Rosenbrock problem
    """
    def __init__(self, input_keys, steepness, num_blocks, penalty_weight=50, output_key="loss"):
        super().__init__()
        self.b_key, self.a_key, self.x_key = input_keys
        self.output_key = output_key
        self.steepness = steepness
        self.num_blocks = num_blocks
        self.penalty_weight = penalty_weight
        self.device = None
        # coefs
        rng = np.random.RandomState(17)
        P = rng.normal(scale=1, size=(3, num_blocks))
        q = rng.normal(scale=1, size=(num_blocks))
        self.P = torch.from_numpy(P).float()
        self.q = torch.from_numpy(q).float()
        

    def forward(self, input_dict):
        """
        forward pass
        """
        # objective function
        obj = self.cal_obj(input_dict)
        # constraints violation
        viol = self.cal_constr_viol(input_dict)
        # penalized loss
        loss = obj + self.penalty_weight * viol
        input_dict[self.output_key] = torch.mean(loss)
        return input_dict

    def cal_obj(self, input_dict):
        """
        calculate objective function
        """
        # get values
        x, a = input_dict[self.x_key], input_dict[self.a_key]
        # x_2i
        x1 = x[:, ::2]
        # x_2i+1
        x2 = x[:, 1::2]
        # objective function
        f = torch.sum((a - x1) ** 2 + self.steepness * (x2 - x1 ** 2) ** 2, dim=1)
        return f

    def cal_constr_viol(self, input_dict):
        """
        calculate constraints violation
        """
        # get values
        x, b = input_dict[self.x_key], input_dict[self.b_key]
        # update device
        if self.device is None:
            self.device = x.device
            self.P = self.P.to(self.device)
            self.q = self.q.to(self.device)
        # inner constraint violation
        lhs_inner = torch.sum(x[:, 1::2], dim=1)
        rhs_inner = self.num_blocks * b[:, 0] / 2
        inner_violation = torch.relu(rhs_inner - lhs_inner)
        # outer constraint violation
        lhs_outer = torch.sum(x[:, ::2] ** 2, dim=1)
        rhs_outer = self.num_blocks * b[:, 0]
        outer_violation = torch.relu(lhs_outer - rhs_outer)
        # lear constraint violation
        lhs = torch.matmul(x[:, 1::2], self.q)
        linear_violation = torch.relu(lhs)
        return inner_violation + outer_violation + linear_violation

In [3]:
import numpy as np
from pyomo import environ as pe
from src.problem.math_solver import abcParamSolver

class msRosenbrock(abcParamSolver):
    def __init__(self, steepness, num_blocks, timelimit=None):
        super().__init__(timelimit=timelimit)
        # create model
        m = pe.ConcreteModel()
        # parameters
        m.b = pe.Param(default=1, mutable=True)
        m.a = pe.Param(pe.RangeSet(0, num_blocks-1), default=1, mutable=True)
        # variables
        m.x = pe.Var(pe.RangeSet(0, num_blocks*2-1), domain=pe.Reals)
        for i in range(num_blocks):
            # integer variables
            m.x[2*i+1].domain = pe.Integers
        # objective
        obj = sum((m.a[i] - m.x[2*i]) ** 2 + \
                   steepness * (m.x[2*i+1] - m.x[2*i] ** 2) ** 2 for i in range(num_blocks))
        m.obj = pe.Objective(sense=pe.minimize, expr=obj)
        # constraints
        m.cons = pe.ConstraintList()
        m.cons.add(sum(m.x[2*i+1] for i in range(num_blocks)) >= num_blocks * m.b / 2)
        m.cons.add(sum(m.x[2*i] ** 2 for i in range(num_blocks)) <= num_blocks * m.b)
        rng = np.random.RandomState(17)
        b = rng.normal(scale=1, size=(3, num_blocks))
        q = rng.normal(scale=1, size=(num_blocks))
        m.cons.add(sum(b[0,i] * m.x[2*i] for i in range(num_blocks)) == 0)
        m.cons.add(sum(b[1,i] * m.x[2*i] for i in range(num_blocks)) == 0)
        m.cons.add(sum(b[2,i] * m.x[2*i] for i in range(num_blocks)) == 0)
        m.cons.add(sum(q[i] * m.x[2*i+1] for i in range(num_blocks)) <= 0)
        # attribute
        self.model = m
        self.params ={"b":m.b, "a":m.a}
        self.vars = {"x":m.x}
        self.cons = m.cons

In [4]:
from scipy.linalg import null_space

class rosenbrockNullSpaceEncoding(nn.Module):
    def __init__(self, num_blocks, input_key, output_key):
        """
        encode equality constraints P x = 0 using null space decomposition.
        """
        super().__init__()
        # size
        self.num_blocks = num_blocks
        # data keys
        self.input_key = input_key
        self.output_key = output_key
        # init encoding
        rng = np.random.RandomState(17)
        P = rng.normal(scale=1, size=(3, self.num_blocks))
        # sepecial solution for equality constraints
        x_s, _, _, _ = np.linalg.lstsq(P, np.zeros(3), rcond=None)
        # null space for equality constraints
        N = null_space(P)
        # to pytorch
        self.x_s = torch.tensor(x_s, dtype=torch.float32).view(1, -1)
        self.N = torch.tensor(N, dtype=torch.float32)
        # init device
        self.device = None

    def forward(self, data):
        # get free parameters
        z = data[self.input_key]
        # batch size
        batch_size = z.shape[0]
        # device
        if z.device != self.device:
            self.device = z.device
            self.x_s = self.x_s.to(self.device)
            self.N = self.N.to(self.device)
        # init x
        x = torch.zeros((batch_size, self.num_blocks*2)).to(self.device)
        # integer part
        x[:, 1::2] = z[:,self.num_blocks-3:]
        # continous part  to encode
        x[:, 0::2] = self.x_s + torch.einsum("bj,ij->bi", z[:,:self.num_blocks-3], self.N)
        data[self.output_key] = x
        # cut off z
        data[self.input_key] = z[:,:self.num_blocks-3]
        return data

### Problem Setting

In [5]:
# init
steepness = 50    # steepness factor
num_blocks = 1000 # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [6]:
# parameters as input data
b_low, b_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
b_train = np.random.uniform(b_low, b_high, (train_size, 1)).astype(np.float32)
b_test  = np.random.uniform(b_low, b_high, (test_size, 1)).astype(np.float32)
b_dev   = np.random.uniform(b_low, b_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [7]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"b":b_train, "a":a_train}, name="train")
data_test = DictDataset({"b":b_test, "a":a_test}, name="test")
data_dev = DictDataset({"b":b_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

### Exact Solver

In [8]:
model = msRosenbrock(steepness, num_blocks, timelimit=60)

### Rounding Classification

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks-3, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b", "a"], ["z"], name="smap")
# linear constraint encode
encoding = rosenbrockNullSpaceEncoding(num_blocks, input_key="z", output_key="x")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks-3)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, equality_encoding=encoding, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, encoding, rnd]).to("cuda")
loss_fn = nmRosenbrock(["b", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 253254.90
Epoch 0, Iters 125, Training Loss: 114294.68, Validation Loss: 36962.46
Epoch 1, Iters 250, Training Loss: 43709.80, Validation Loss: 50492.13
Epoch 2, Iters 375, Training Loss: 29650.02, Validation Loss: 23970.49
Epoch 3, Iters 500, Training Loss: 25474.91, Validation Loss: 19786.68
Epoch 4, Iters 625, Training Loss: 22803.13, Validation Loss: 26694.25
Epoch 5, Iters 750, Training Loss: 21417.09, Validation Loss: 15319.05
Epoch 6, Iters 875, Training Loss: 23052.28, Validation Loss: 23887.53
Epoch 7, Iters 1000, Training Loss: 25260.71, Validation Loss: 35522.57
Epoch 8, Iters 1125, Training Loss: 26313.61, Validation Loss: 12677.59
Epoch 9, Iters 1250, Training Loss: 21276.18, Validation Loss: 18483.83
Epoch 10, Iters 1375, Training Loss: 21925.81, Validation Loss: 46870.11
Epoch 11, Iters 1500, Training Loss: 21236.70, Validation Loss: 11552.81
Epoch 12, Iters 1625, Training Loss: 15106.32, Validation Loss: 9705.02
Epoch 13, Iters 1750, T

In [13]:
params, sols, objvals, mean_viols, max_viols, num_viols, elapseds = [], [], [], [], [], [], []
for b, a in tqdm(list(zip(b_test, a_test))):
    # data point as tensor
    datapoints = {"b": torch.tensor(np.array([b]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    viol = model.cal_violation()
    mean_viols.append(np.mean(viol))
    max_viols.append(np.max(viol))
    num_viols.append(np.sum(viol > 1e-6))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param": params,
                    "Sol": sols,
                    "Obj Val": objvals,
                    "Mean Violation": mean_viols,
                    "Max Violation": max_viols,
                    "Num Violations": num_viols,
                    "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Num Violations"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.49it/s]


           Obj Val  Mean Violation  Max Violation  Num Violations  \
count   100.000000    1.000000e+02     100.000000      100.000000   
mean   6712.276269    4.690160e-07       0.000003        0.220000   
std     928.426149    9.320602e-07       0.000006        0.416333   
min    4837.228222    0.000000e+00       0.000000        0.000000   
25%    6131.733952    0.000000e+00       0.000000        0.000000   
50%    6673.853595    0.000000e+00       0.000000        0.000000   
75%    7350.279066    0.000000e+00       0.000000        0.000000   
max    9779.152363    4.613341e-06       0.000028        1.000000   

       Elapsed Time  
count    100.000000  
mean       0.005424  
std        0.001621  
min        0.003757  
25%        0.004532  
50%        0.004980  
75%        0.005868  
max        0.015712  
Number of infeasible solution: 22


### Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks-3, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b", "a"], ["z"], name="smap")
# linear constraint encode
encoding = rosenbrockNullSpaceEncoding(num_blocks, input_key="z", output_key="x")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks-3)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, equality_encoding=encoding, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, encoding, rnd]).to("cuda")
loss_fn = nmRosenbrock(["b", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Iters 0, Validation Loss: 226793.92
Epoch 0, Iters 125, Training Loss: 101621.04, Validation Loss: 87224.96
Epoch 1, Iters 250, Training Loss: 38232.27, Validation Loss: 49140.71
Epoch 2, Iters 375, Training Loss: 43812.98, Validation Loss: 39851.13
Epoch 3, Iters 500, Training Loss: 34620.54, Validation Loss: 18236.76
Epoch 4, Iters 625, Training Loss: 28061.13, Validation Loss: 25862.74
Epoch 5, Iters 750, Training Loss: 20003.14, Validation Loss: 38530.15
Epoch 6, Iters 875, Training Loss: 22773.50, Validation Loss: 17404.29
Epoch 7, Iters 1000, Training Loss: 22601.10, Validation Loss: 29516.13
Epoch 8, Iters 1125, Training Loss: 17305.48, Validation Loss: 14817.75
Epoch 9, Iters 1250, Training Loss: 13608.44, Validation Loss: 8570.97
Epoch 10, Iters 1375, Training Loss: 13466.18, Validation Loss: 14038.11
Epoch 11, Iters 1500, Training Loss: 11139.57, Validation Loss: 27492.76
Epoch 12, Iters 1625, Training Loss: 13660.12, Validation Loss: 10267.01
Epoch 13, Iters 1750, T

In [18]:
params, sols, objvals, mean_viols, max_viols, num_viols, elapseds = [], [], [], [], [], [], []
for b, a in tqdm(list(zip(b_test, a_test))):
    # data point as tensor
    datapoints = {"b": torch.tensor(np.array([b]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    viol = model.cal_violation()
    mean_viols.append(np.mean(viol))
    max_viols.append(np.max(viol))
    num_viols.append(np.sum(viol > 1e-6))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param": params,
                    "Sol": sols,
                    "Obj Val": objvals,
                    "Mean Violation": mean_viols,
                    "Max Violation": max_viols,
                    "Num Violations": num_viols,
                    "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Num Violations"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.54it/s]


           Obj Val  Mean Violation  Max Violation  Num Violations  \
count   100.000000      100.000000     100.000000      100.000000   
mean   5922.541825        0.072536       0.435215        0.260000   
std     453.377995        0.725354       4.352122        0.524549   
min    4967.965728        0.000000       0.000000        0.000000   
25%    5672.595515        0.000000       0.000000        0.000000   
50%    5895.431849        0.000000       0.000000        0.000000   
75%    6119.950248        0.000000       0.000000        0.000000   
max    7250.742694        7.253538      43.521228        3.000000   

       Elapsed Time  
count    100.000000  
mean       0.005048  
std        0.000875  
min        0.003525  
25%        0.004514  
50%        0.004998  
75%        0.005324  
max        0.007976  
Number of infeasible solution: 23
